In [ ]:
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import math


In [ ]:
def get_image(image_path):
    return cv2.imread(image_path)

In [ ]:
image = get_image('./Dataset/game1/Clip1/0000.jpg')

In [ ]:
plt.imshow(image)

In [ ]:
# Mouse callback function to collect points
points=[]
cv2.imshow('Select Points', image)
def get_points(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        points.append((x, y))
        print("Point selected: ({}, {})".format(x, y))

# Set the mouse callback
cv2.setMouseCallback('Select Points', get_points)

# Wait for the user to select points
cv2.waitKey(0)

# Close the image window
cv2.destroyAllWindows()

# Print the selected points
print("Selected Points:", points)

In [ ]:
# top left corner, top right corner, bottom right corner, bottom left corner, pole left, pole right
points=[(420, 165), (860, 165), (1090, 530), (200, 530), (291, 230), (992, 230)]
#plot points on frame
for point in points:
    cv2.circle(image, point, 5, (0, 0, 255), -1)
cv2.imshow('points',image)
cv2.waitKey(0)

# Close the image window
cv2.destroyAllWindows()



In [ ]:
#true coords
# top left corner, top right corner, bottom right corner, bottom left corner, pole left, pole right
true_coords = [(0,23.77, 0), (10.97,23.77,  0), (10.97,0, 0),(0,0,0), (-0.91, 5.485,1.07), (11.42,5.485,1.07)]

In [ ]:
A = []
for i in range(6):
    X, Y, Z = true_coords[i]
    u, v = points[i]
    A.append([X, Y, Z, 1, 0, 0, 0, 0, -u*X, -u*Y, -u*Z, -u])
    A.append([0, 0, 0, 0, X, Y, Z, 1, -v*X, -v*Y, -v*Z, -v])

In [ ]:
A = np.array(A)


In [ ]:
image.shape

In [ ]:
# svd of A
U, S, V = np.linalg.svd(A)

In [ ]:
V.shape

In [ ]:
# get last column of V
p = V[-1, :]

In [ ]:
p.shape

In [ ]:
# reshape to 
M = np.reshape(p, (3, 4))

In [ ]:
M

In [ ]:
a1 = M[0, :3]
a2 = M[1, :3]
a3 = M[2, :3]
b1 = M[0, 3]
b2 = M[1, 3]
b3 = M[2, 3]

In [ ]:
rho = 1 / np.linalg.norm(a3)
cx = rho**2 * np.dot(a1, a3)
cy = rho**2 * np.dot(a2, a3)
cos_theta = -np.dot(np.cross(a1, a3), np.cross(a2, a3)) / (np.linalg.norm(np.cross(a1, a3)) * np.linalg.norm(np.cross(a2, a3)))
sin_tetha = np.sqrt(1 - cos_theta**2)
alpha = rho**2 * np.linalg.norm(np.cross(a1, a3)) * sin_tetha
beta = rho**2 * np.linalg.norm(np.cross(a2, a3)) * sin_tetha

In [ ]:
cy

In [ ]:
K = np.array([[alpha, -alpha* (cos_theta/sin_tetha), cx], [0, beta/ sin_tetha, cy], [0, 0, 1]])
#K = np.array([[alpha, 0, cx], [0, alpha, cy], [0, 0, 1]])

In [ ]:
K

In [ ]:
# get R and T
r1 = (np.cross(a2, a3) / np.linalg.norm(np.cross(a2, a3)))
r3= rho * a3
r2 = np.cross(r3, r1)

In [ ]:
b = np.array([b1, b2, b3])
t = rho* np.linalg.inv(K).dot(b)

In [ ]:
# use K R and t to project the points in 3d and show with matplotlib
points_2d = np.array(points)
# add 1 to the points
points_2d = np.hstack((points_2d, np.ones((6, 1))))
points_2d


In [ ]:
R = np.array([r1, r2, r3])

In [ ]:
t = t.reshape(3, 1)
t

In [ ]:
points_normalized = np.linalg.inv(K) @ points_2d.T
points_3d = (np.linalg.inv(R) @ (points_normalized - t)).T


In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

ax.scatter(points_3d[:, 0], points_3d[:, 1], points_3d[:, 2], c='r', marker='o')

ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')

plt.show()

In [ ]:
x = points_3d[:, 0]
y = points_3d[:, 1]
z = points_3d[:, 2]

# Create Plotly figure
fig = go.Figure(data=[go.Scatter3d(x=x, y=y, z=z, mode='markers', marker=dict(size=5))])

# Set axes labels
fig.update_layout(scene=dict(xaxis_title='X', yaxis_title='Y', zaxis_title='Z'))

# Show plot
fig.show()

In [ ]:
rows,cols,ch = image.shape

pts1 = np.float32([[420, 165], [860, 165], [1090, 530], [200, 530]])

ratio=0.02
cardH=math.sqrt((pts1[2][0]-pts1[1][0])*(pts1[2][0]-pts1[1][0])+(pts1[2][1]-pts1[1][1])*(pts1[2][1]-pts1[1][1]))
cardW=ratio*cardH
pts2 = np.float32([[0,2377], [1097,2377], [1097,0],[0,0]])

M = cv2.getPerspectiveTransform(pts1,pts2)

offsetSize=20
transformed = np.zeros((int(cardW+offsetSize), int(cardH+offsetSize)), dtype=np.uint8);
dst = cv2.warpPerspective(image, M, (2000,2500))

plt.subplot(121),plt.imshow(image),plt.title('Input')
plt.subplot(122),plt.imshow(dst),plt.title('Output')
plt.show()

In [ ]:
rectified_pts = cv2.perspectiveTransform(pts1.reshape(-1, 1, 2), M)

# Print the coordinates of original points in the new rectified image
for point in rectified_pts:
    x, y = point[0]
    print("Original Point: ({}, {}) -> New Rectified Image: ({}, {})".format(pts1[0][0], pts1[0][1], x, y))

In [ ]:
point1= np.float32([420, 165, 1])
corenr=np.dot(M , point1)
corner= corenr/corenr[2]
corner

In [ ]:
rectified_pts

In [ ]:
d_3pts = []
for point in rectified_pts:
    x, y = point[0]
    point_3d = np.array([x, y, 1])
    point_3d = np.dot(np.linalg.inv(M), point_3d)
    d_3pts.append(point_3d)
d_3pts
    

In [ ]:
d_3pts = np.array(d_3pts)

In [ ]:
x = d_3pts[:, 0]
y = d_3pts[:, 1]
z = d_3pts[:, 2]

# Create Plotly figure
fig = go.Figure(data=[go.Scatter3d(x=x, y=y, z=z, mode='markers', marker=dict(size=5))])
#connect the 4 points
fig.add_trace(go.Scatter3d(x=[x[0], x[1]], y=[y[0], y[1]], z=[z[0], z[1]], mode='lines', name='top'))
fig.add_trace(go.Scatter3d(x=[x[1], x[2]], y=[y[1], y[2]], z=[z[1], z[2]], mode='lines', name='right'))
fig.add_trace(go.Scatter3d(x=[x[2], x[3]], y=[y[2], y[3]], z=[z[2], z[3]], mode='lines', name='bottom'))
fig.add_trace(go.Scatter3d(x=[x[3], x[0]], y=[y[3], y[0]], z=[z[3], z[0]], mode='lines', name='left'))

# Set axes labels
fig.update_layout(scene=dict(xaxis_title='X', yaxis_title='Y', zaxis_title='Z'))

# Show plot
fig.show()

In [ ]:
# calcula the distance between the 2 points
def distance_3d(point1, point2):
    return np.linalg.norm(point1[:2] - point2[:2])
for i in range(4):
    for j in range(i+1, 4):
        print("Distance between point {} and point {} is: {}".format(i, j, distance_3d(d_3pts[i], d_3pts[j])))